In [1]:
import pandas as pd
import cv2
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import seaborn as sns
import os

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

2023-03-25 21:06:19.706903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Train: cat = 3000 images, dog = 3000 images
#Val : cat = 1500 iamges, dog = 1000 images

base_dir = r'/Users/brandontan/a_data_truncated'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')



In [3]:

train_data = ImageDataGenerator(rescale = 1./255,
                                shear_range = 0.2,
                                zoom_range = 0.2,
                                horizontal_flip = True)

train_gen = train_data.flow_from_directory(train_dir,
                                           target_size = (128, 128),
                                           batch_size = 64,
                                           class_mode = 'binary')

#Val and test only need rescaling
val_test_data = ImageDataGenerator(rescale = 1./255)

val_gen = val_test_data.flow_from_directory(val_dir,
                                           target_size = (128, 128),
                                           batch_size = 64,
                                           class_mode = 'binary')


Found 6002 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
#cnn.add(Conv2D(filters, (kernel size), inputshape))

cnn = Sequential()
cnn.add(Conv2D(32, (3,3), input_shape = (128, 128, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size = (2,2)))

cnn.add(Conv2D(32, (3,3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size = (2,2)))

cnn.add(Conv2D(64, (3,3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size = (2,2)))

#Fully connected layer
#Flatten change the dimension to 1d
cnn.add(Flatten())
cnn.add(Dense(128))
cnn.add(Activation('relu'))
cnn.add(Dense(1))
cnn.add(Activation('sigmoid'))

cnn.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics = ['accuracy'])


2023-03-25 21:06:38.205289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#Early stopping method - prevent overfitting
early_stop = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1)

In [6]:
#Training begins
cnn.fit(train_gen, validation_data = val_gen, epochs = 1, callbacks = [early_stop])

94/94 [==============================] - 76s 783ms/step - loss: 0.6918 - accuracy: 0.5420 - val_loss: 0.6796 - val_accuracy: 0.5840


In [10]:
from joblib import dump
import joblib

In [11]:
dump(cnn, './../mlmodel/cnn_model.joblib')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_2
.........vars
......activation_3
.........vars
......activation_4
.........vars
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......conv2d_2
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
......max_pooling2d_2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........3
.........4
.....

['./../mlmodel/cnn_model.joblib']

cnnaaa = joblib.load('/Users/brandontan/Desktop/FYP/num2/FYP-23-S1-05/fypMain/whatisthis/MLmodel/cnn_model.joblib')

cnnaaa

modell = cnnaaa.predict(val_gen)
modell